<a href="https://colab.research.google.com/github/BluBloos/3D-Hand-Tracking/blob/main/src/HandTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [ ]:
# RUN THIS BLOCK ONLY WHEN IN COLAB

!echo "Initializing github repository"
!ls -la
!rm -r .config/
!rm -r sample_data/
!git clone https://github.com/BluBloos/QMIND2021-2022/ .

In [ ]:
# ALWAYS RUN THIS BLOCK, COLAB OR NOT

# Download updated project from Github.
!git pull

##### HANDLE DIFFS WHEN RUNNING IN COLAB #####
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
print("In Colab:", IN_COLAB)
import sys
if (IN_COLAB):
  sys.path.insert(1, '/content/src/')
##### HANDLE DIFFS WHEN RUNNING IN COLAB #####

########### TEST GPU AND RAM OF COLLAB INSTANCE ###########
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')
########### TEST GPU AND RAM OF COLLAB INSTANCE ###########

######### EXTERNAL LIBRARIES #########
import os
import pickle
import matplotlib.pyplot as plt
import imageio
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, MaxPool2D
from tensorflow.keras import Model
print("TensorFlow version:", tf.__version__)
#NOTE: Good resource. -> https://www.tensorflow.org/tutorials/quickstart/advanced
import cv2 # opencv, for image resizing.
######### EXTERNAL LIBRARIES #########

############## HELPER FUNCTIONS ############## 
# NOTE(Noah): Stole this function from Stackoverflow :)
def rgb2gray(rgb):
    return np.expand_dims(np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]), axis=2)
def resize(img, size):
    return cv2.resize(img, dsize=(size, size), interpolation=cv2.INTER_CUBIC)
############## HELPER FUNCTIONS ############## 

# MODEL LOADING

In [ ]:
# TODO(Noah): Get the MANO folders hosted in GCS so that this works again.
#   We note that this cost was tested and is in full working order, so 
#   the only thing not working is the lack of existence of MANO_DIR. 

# Setup some params.
IMAGE_SIZE = 224
GRAYSCALE = False
IMAGE_CHANNELS = 1 if GRAYSCALE else 3
BATCH_SIZE = 32
MANO_DIR = "mano_v1_2" if IN_COLAB else "../mano_v1_2"

from mobilehand import MAKE_MOBILE_HAND
from mobilehand_lfuncs import LOSS_3D

MOBILE_HAND = MAKE_MOBILE_HAND(IMAGE_SIZE, IMAGE_CHANNELS, BATCH_SIZE, MANO_DIR)

# INTEGRATION TEST
input_test = tf.random.uniform(shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
input_test = tf.cast(input_test, tf.float32)
output_test = MOBILE_HAND(input_test)
print(output_test)

# The lower training loop assumes that the model is set as such.
model = MOBILE_HAND

# The lower training loop also assumes that we have the loss function set like so.
loss_fn = lambda pred, gt : LOSS_3D(pred,gt) 

# DATA LOADING

In [ ]:
from matplotlib.image import imread

# NOTE(Noah): gcs code will on work on the colab as this code is Ubuntu specific.
# I attempted to install gcsfuse on my macOS machine, but I have a version that is too new.
# also, gcsfuse simply does not work on Windows.
#
# gcsfuse is actually beta software.
if IN_COLAB:
  from google.colab import auth
  auth.authenticate_user()

  # we know that we are on an Ubuntu machine.
  # Thus, installing gcsfuse will be done via the Ubuntu instructions.
  # https://github.com/GoogleCloudPlatform/gcsfuse/blob/master/docs/installing.md#ubuntu-and-debian-latest-releases
  !echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
  !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
  
  # -y in apt-get will assume "yes" as the answer to all prompts.
  # -q in apt-get will make things "quiet" for us. Nice!
  !sudo apt-get -y -q update
  !sudo apt-get -y -q install gcsfuse

  !mkdir -p data
  !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 shd_final data

def download_image(path):
  image = imageio.imread(path)
  _image = image.astype('float32')
  if GRAYSCALE:
      _image = rgb2gray(_image / 255)
  else:
      _image = _image / 255
  _image = resize(_image, IMAGE_SIZE)
  return _image

# TODO(Noah): Reimplement the code that sets up SH_RHD.
gcs_path = 'data' if IN_COLAB else '../SH_RHD'
train_list = os.listdir(os.join(gcs_path, "training/color"))
eval_list = os.listdir(os.join(gcs_path, "evaluation/color"))

# numpy "buckets" that we will use to load things in.
x_train = np.zeros( (32, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) )
y_train = np.zeros( (32, 21, 3) )
x_test = np.zeros( (32, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS) ) 
y_test = np.zeros( (32, 21, 3) )
TRAIN_IMAGES = (len(train_list) // 32) * 32
TEST_IMAGES = (len(eval_list) // 32) * 32

# TRAINING LOOP

In [ ]:
class StupidSimpleLossMetric():
    def __init__(self):
        self.losses = [] # empty python array 
    def __call__(self, loss):
        self.losses.append(loss)
    def result(self):
        return sum(self.losses) / len(self.losses)
    def reset_states(self):
        self.losses = []

optimizer = tf.keras.optimizers.Adam() # defaults should work just fine
train_loss = StupidSimpleLossMetric()
test_loss = StupidSimpleLossMetric()

# Loss function unit test
input = tf.zeros([1, 21,3])  # mock pred of all zeros
label = np.expand_dims(y_train[0], axis=0)
loss = loss_fn(input, label) 
print('Loss for pred of all zeros', loss.numpy())
#loss2 = loss_fn(label, label)
#print('Loss for perfect prediction', loss2.numpy())
input2 = tf.ones([1, 21, 3])
loss3 = loss_fn(input2, label)
print('Loss for pred of all ones', loss3.numpy())

@tf.function
def train_step(input, gt):
    with tf.GradientTape() as tape:
        predictions = model(input)
        #loss = loss_func(predictions, segmentation_masks)
        #loss = np.dot(tf.reshape(segmentation_masks, [102400], tf.reshape(predictions, [102400])
        loss = loss_fn(predictions, gt)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss
    #train_accuracy(labels, predictions)
  
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  return loss_fn(predictions, labels)
  #test_accuracy(labels, predictions)

In [ ]:
checkpoint_path = MANO_DIR = "checkpoints/" if IN_COLAB else "../checkpoints/"
!mkdir $checkpoint_path

last_checkpoint = -1
if (last_checkpoint > -1):
  model.load_weights(os.join(checkpoint_path, "cp-{last_checkpoint:04d}.ckpt"))

EPOCHS = 10 # sure...

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  print("Epoch", epoch)
  start = time.time()
  train_loss.reset_states()
  #train_accuracy.reset_states()
  test_loss.reset_states()
  #test_accuracy.reset_states()
  
  for i in range(TRAIN_IMAGES / 32):
    for j in range(32):
      train_image = download_image(os.path.join(gcs_path, 'training/color', train_list[j + i * 32]))
      x_train[j,:,:,:] = train_image
     
    # TODO(Data Team): Load in the annotations.
    x_train = x_train.astype('float32')
    y_train = y_train.astype('float32')

    loss = train_step(x_train, y_train)
    train_loss(loss.numpy())

  for i in range(TEST_IMAGES / 32):
    for j in range(32):
      eval_image = download_image(os.path.join(gcs_path, 'evaluation/color', eval_list[j + i * 32]))
      x_test[j,:,:,:] = eval_image
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')

    loss_test = test_step(x_test, y_test)
    test_loss(loss.numpy())

  end = time.time()

  print(
    f'Epoch {epoch + 1}, '
    f'Time {end-start} s'
    f'Loss: {train_loss.result()}, '
    f'Test Loss: {test_loss.result()}, '
  )

  # Save the model parameters
  if (epoch % 5 == 0):
    checkpoint_filepath = os.join(checkpoint_path, "cp-{epoch:04d}.ckpt")
    model.save_weights(checkpoint_filepath)